In [8]:
import os
from datetime import datetime
from dotenv import load_dotenv

#--------Google Drive Integration--------#
# from google.colab import drive, userdata
# This gives Colab access to your files in Google Drive.
# drive.mount('/content/drive')
# 'GITHUB_USERNAME' and 'GITHUB_TOKEN' saved as secrets in Colab.
# GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
# GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# REPOSITORY_NAME = 'PyNucleus-Model' # Your repository name
# NOTEBOOK_DRIVE_PATH = "/content/drive/MyDrive/PyNucleus Project/Capstone Project.ipynb"


#--------Cursor Integration--------#
# Load environment variables from .env file
load_dotenv()

# Get GitHub credentials from environment variables
GITHUB_USERNAME = os.getenv('GITHUB_USERNAME')
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

# Print to verify the variables are loaded (remove this in production)
print(f"Username: {GITHUB_USERNAME}")
print(f"Token: {GITHUB_TOKEN[:4]}...") # Only print first 4 chars of token for security

# Repository information
REPOSITORY_NAME = 'PyNucleus-Model'
NOTEBOOK_REPO_FILENAME = "Capstone Project.ipynb"
LOG_FILENAME = "update_log.txt"

# Pull latest changes from GitHub
print("Pulling latest changes from GitHub...")
!git pull https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

print("Repository is up to date!")

Username: Saytor20
Token: ghp_...
Pulling latest changes from GitHub...
From https://github.com/Saytor20/PyNucleus-Model
 * branch            main       -> FETCH_HEAD
Already up to date.
Repository is up to date!


In [9]:
# # Clone the Repository
# repo_path = f'/content/{REPOSITORY_NAME}'
# !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git {repo_path}

# # Change the current working directory to the repository
# # All subsequent commands will run from inside the repo folder.
# os.chdir(repo_path)

# # Add a new line to your log file with the current date and time.
# log_message = f"Notebook saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
# with open(LOG_FILENAME, "a") as f:
#     f.write(log_message + "\n")
# print(f"Updated '{LOG_FILENAME}'")

# # Copy the latest version of notebook from Drive into the cloned repo.
# !cp "{NOTEBOOK_DRIVE_PATH}" "{NOTEBOOK_REPO_FILENAME}"
# print(f"Copied '{NOTEBOOK_REPO_FILENAME}' from Google Drive.")

# # Git identity for commenting
# !git config user.name "{GITHUB_USERNAME}"
# !git config user.email "{GITHUB_USERNAME}@users.noreply.github.com"

# **Data Ingestion and Preprocessing for RAG**

In [19]:
# Data Ingestion and Preprocessing for RAG
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the path to your data (current directory)
DATA_DIR = os.getcwd()

# Load all.txt files from the current directory
print(f"Loading documents from: {DATA_DIR}")
loader = DirectoryLoader(DATA_DIR, glob="*.txt", loader_cls=TextLoader, show_progress=True)
documents = loader.load()

print(f"\nLoaded {len(documents)} documents.")
for i, doc in enumerate(documents):
    print(f"  - Document {i+1} Source: {os.path.basename(doc.metadata.get('source', 'N/A'))}")

Loading documents from: /Users/mohammadalmusaiteer/PyNucleus-Model


100%|██████████| 2/2 [00:00<00:00, 491.31it/s]


Loaded 2 documents.
  - Document 1 Source: mcp_basics.txt
  - Document 2 Source: feasibility_factors.txt


In [25]:
#2. Text Chunking

# Define chunking parameters
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

# Create the text splitter with these parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Apply the splitter to the loaded documents
chunked_documents = text_splitter.split_documents(documents)

print(f"\nSplit {len(documents)} documents into {len(chunked_documents)} chunks.")



Split 2 documents into 4 chunks.


In [29]:
# --- 4. Inspect the Results ---
import numpy as np
# --- NEW: Statistical Analysis of Chunks ---
print("\n--- Statistical Analysis & Quality Check ---")

# Calculate the lengths of all chunks
chunk_lengths = [len(chunk.page_content) for chunk in chunked_documents]

# Calculate and print key statistics
total_chunks = len(chunk_lengths)
min_size = np.min(chunk_lengths)
max_size = np.max(chunk_lengths)
avg_size = np.mean(chunk_lengths)
std_dev = np.std(chunk_lengths)

print(f"Total Chunks: {total_chunks}")
print(f"Minimum Chunk Size: {min_size} characters")
print(f"Maximum Chunk Size: {max_size} characters")
print(f"Average Chunk Size: {avg_size:.2f} characters")
print(f"Standard Deviation of Chunk Size: {std_dev:.2f}")

# --- Automated Quality Feedback ---

# 1. Check for high variation in chunk size
# A high standard deviation suggests inconsistent chunking.
if std_dev > 150:
    print(f"\n[WARNING] High chunk size variation detected (Std Dev: {std_dev:.2f}).")
    print("  > This suggests documents may have irregular structures (e.g., many short lines or lists).")
    print("  > Resulting chunks may have inconsistent levels of context.")

# 2. Check for and count potentially "orphaned" or very small chunks
small_chunk_threshold = CHUNK_SIZE * 0.20 # Chunks smaller than 20% of the target size
small_chunk_count = sum(1 for length in chunk_lengths if length < small_chunk_threshold)

if small_chunk_count > 0:
    # This check is more specific than just looking at the absolute minimum.
    print(f"\n[ADVISORY] Found {small_chunk_count} chunks smaller than {small_chunk_threshold} characters.")
    print(f"  > The smallest chunk is {min_size} characters.")
    print("  > These small chunks might lack sufficient context and could clutter search results.")
    print("  > Consider cleaning the source documents or adjusting the chunking separators.")

# Add a success message if no issues are flagged
if std_dev <= 150 and small_chunk_count == 0:
    print("\n[INFO] Chunking statistics appear healthy. Sizes are consistent.")


# --- Manual Inspection of Sample Chunks ---
# (This part remains the same)
print("\n--- Sample Chunk Preview ---")
# Print the first few chunks to get a feel for their content and structure
for i, chunk in enumerate(chunked_documents[:3]): # Print first 3 chunks
    chunk_source = os.path.basename(chunk.metadata.get('source', 'N/A'))
    print(f"\n--- Chunk {i+1} (Source: {chunk_source}, Length: {len(chunk.page_content)} chars) ---")
    print(chunk.page_content)


print("\n\nData ingestion and preprocessing is complete. The 'chunked_documents' are ready for the next stage (embedding).")


--- Statistical Analysis & Quality Check ---
Total Chunks: 4
Minimum Chunk Size: 126 characters
Maximum Chunk Size: 472 characters
Average Chunk Size: 322.00 characters
Standard Deviation of Chunk Size: 143.16

[INFO] Chunking statistics appear healthy. Sizes are consistent.

--- Sample Chunk Preview ---

--- Chunk 1 (Source: mcp_basics.txt, Length: 472 chars) ---
Modular Chemical Plants (MCPs) represent a paradigm shift in chemical process engineering. They involve constructing plants from standardized, pre-fabricated modules built off-site. This approach significantly reduces on-site construction time and costs compared to traditional stick-built plants. Key advantages include faster deployment, scalability, and potentially lower capital expenditure. However, module transportation and site integration require careful planning

--- Chunk 2 (Source: mcp_basics.txt, Length: 126 chars) ---
. MCPs are particularly suited for remote locations or projects with uncertain market demands, all

# **Data Scrapping**

In [36]:
# --- 1. Installation (run once in your environment) ---

import os
from googlesearch import search
from trafilatura import fetch_url, extract

# --- 2. Configuration ---
KEYWORDS = ["modular chemical plants", "MCPs feasibility", "pre-fabricated chemical modules"]
NUM_RESULTS = 5
OUTPUT_PATH = os.path.join("data_sources", "web_scraped_mcp_data.txt")

# --- 3. Search + Scrape ---
query = " ".join(KEYWORDS)
try:
    urls = list(search(query, num_results=NUM_RESULTS, lang="en"))
    print(f"🔍 Found {len(urls)} URLs for '{query}'.")
except Exception as e:
    print(f"❌ Search error: {e}")
    urls = []

scraped_contents = []
success_count = 0
fail_count = 0

for url in urls:
    print(f"  ▶ Scraping: {url}")
    raw = fetch_url(url)
    if not raw:
        print("    • ❌ Download failed")
        fail_count += 1
        continue

    text = extract(raw, include_comments=False, include_tables=False)
    if text:
        scraped_contents.append(f"--- Content from: {url} ---\n{text}\n\n")
        success_count += 1
        print("    • ✅ Extracted main text")
    else:
        print("    • ⚠️ No main text found")
        fail_count += 1

# --- 4. Save + Metrics ---
if scraped_contents:
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
        f.write("".join(scraped_contents))

    total_chars = sum(len(chunk) for chunk in scraped_contents)
    total_words = sum(len(chunk.split()) for chunk in scraped_contents)

    print(f"\n✅ Saved {success_count} pages to '{OUTPUT_PATH}'.")
    print("📊 Key Metrics:")
    print(f"   • URLs Found    : {len(urls)}")
    print(f"   • Successful    : {success_count}")
    print(f"   • Failed        : {fail_count}")
    print(f"   • Total Chars   : {total_chars}")
    print(f"   • Total Words   : {total_words}")
else:
    print("\n❌ No text was scraped; nothing to save.")


🔍 Found 5 URLs for 'modular chemical plants MCPs feasibility pre-fabricated chemical modules'.
  ▶ Scraping: /search?num=7
    • ❌ Download failed
  ▶ Scraping: https://www.hm-ec.com/blog-posts/modular-process-plants-hm
    • ✅ Extracted main text
  ▶ Scraping: https://www.tce.co.in/blogs/Modularization%20of%20chemical%20plants.pdf
    • ⚠️ No main text found
  ▶ Scraping: https://dechema.de/Cost_Engineering_for_Modular_Plants/_/2024.11.14%20Cost%20Engineering%20for%20Modular%20Plants%20FINAL.pdf
    • ⚠️ No main text found
  ▶ Scraping: https://aris.iaea.org/publications/smr_book_2020.pdf
    • ❌ Download failed

✅ Saved 1 pages to 'data_sources/web_scraped_mcp_data.txt'.
📊 Key Metrics:
   • URLs Found    : 5
   • Successful    : 1
   • Failed        : 4
   • Total Chars   : 5861
   • Total Words   : 803


# This is the last cell of the code

In [37]:
# Simple GitHub update function
def update_github():
    !git add .
    !git commit -m "Update: Adding all files to repository"
    !git push origin main
    print("All files pushed to GitHub successfully!")

# To use it, just run:
update_github()

[main d3fa170] Update: Adding all files to repository
 2 files changed, 198 insertions(+), 120 deletions(-)
 create mode 100644 data_sources/web_scraped_mcp_data.txt
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 8 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 5.49 KiB | 5.49 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Saytor20/PyNucleus-Model.git
   2702bc3..d3fa170  main -> main
All files pushed to GitHub successfully!
